# XGB Boosting Over Residuals - CV 0.05595
This is a starter notebook demonstrating "boosting over residuals". For the original dataset, @siukeitin (Kaggle user broccoli beef) published the optimal Bayesian solution [here][2]. (This is based on the original data's generation formula [here][1]).

For this month's playground competition, Kaggle created synthetic data from this original data. Therefore the signal has been altered and augmented. Hence the optimal Bayesian solution is no longer optimal for this month's playground competition's data.

None-the-less, we can begin with this optimal solution and have XGB learn how to improve it. So, instead of training with `target`, we will train our XGB with `target minus optimal solution` (i.e. the residual). XGB will learn to predict this residual aka "boost over residuals"!

Discussion about this notebook is [here][3]

[1]: https://www.kaggle.com/code/ianktoo/simulated-road-accident-data-generator
[2]: https://www.kaggle.com/competitions/playground-series-s5e10/discussion/609994#3296622
[3]: https://www.kaggle.com/competitions/playground-series-s5e10/discussion/610828

# Load Train, Test, Original

In [1]:
import pandas as pd, numpy as np

train = pd.read_csv("/kaggle/input/playground-series-s5e10/train.csv")
print("Train shape:", train.shape )
train.head()

Train shape: (517754, 14)


,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [2]:
test = pd.read_csv("/kaggle/input/playground-series-s5e10/test.csv")
test['accident_risk'] = 0.5
print("Test shape:", test.shape )
test.head()

Test shape: (172585, 14)


,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,517754,highway,2,0.34,45,night,clear,True,True,afternoon,True,True,1,0.5
1,517755,urban,3,0.04,45,dim,foggy,True,False,afternoon,True,False,0,0.5
2,517756,urban,2,0.59,35,dim,clear,True,False,afternoon,True,True,1,0.5
3,517757,rural,4,0.95,35,daylight,rainy,False,False,afternoon,False,False,2,0.5
4,517758,highway,2,0.86,35,daylight,clear,True,False,evening,False,True,3,0.5


In [3]:
orig = []
for k in [2,10,100]:
    df = pd.read_csv(f"/kaggle/input/simulated-roads-accident-data/synthetic_road_accidents_{k}k.csv")
    orig.append(df)
orig = pd.concat(orig,axis=0)
orig['id'] = np.arange(len(orig))+test['id'].max()+1
orig = orig[ train.columns ] 
print("Original data shape:", orig.shape )
orig.head()

Original data shape: (112000, 14)


,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,690339,rural,2,0.72,60,daylight,clear,True,False,afternoon,False,False,2,0.37
1,690340,highway,4,0.95,45,daylight,foggy,False,True,evening,False,True,1,0.40
2,690341,rural,1,0.72,25,night,rainy,False,False,evening,True,False,1,0.55
3,690342,rural,4,0.86,70,dim,foggy,True,False,morning,True,True,1,0.56
4,690343,highway,1,0.00,60,night,rainy,True,True,morning,True,True,3,0.54


In [4]:
combine = pd.concat([train,test,orig],axis=0,ignore_index=True)
print("Combine shape:", combine.shape )
combine.head()

Combine shape: (802339, 14)


,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [6]:
import os
files = os.listdir("/kaggle/input/simulated-roads-accident-data")
print(files)

['synthetic_road_accidents_10k.csv', 'synthetic_road_accidents_2k.csv', 'synthetic_road_accidents_100k.csv']


추가 데이터를 가져옴  

# Feature Engineer
We will preprocess/feature engineer the following:
* Add @siukeitin optimal original data solution (from [here][1])
* Label encode the categorical features
* Target encode all features using original data targets

[1]: https://www.kaggle.com/competitions/playground-series-s5e10/discussion/609994#3296622

In [ ]:
FEATURES = list( orig.columns[1:-1] )
TARGET = orig.columns[-1]
print(f"Features: {FEATURES}, Target: '{TARGET}'")

FEATURES: 인덱스와 정답을 제거  

## - Add New Feature!

In [ ]:
# https://www.kaggle.com/competitions/playground-series-s5e10/discussion/609994#3296622
import scipy

def f(X):
    return \
    0.3 * X["curvature"] + \
    0.2 * (X["lighting"] == "night").astype(int) + \
    0.1 * (X["weather"] != "clear").astype(int) + \
    0.2 * (X["speed_limit"] >= 60).astype(int) + \
    0.1 * (X["num_reported_accidents"] > 2).astype(int)

def clip(f):
    def clip_f(X):
        sigma = 0.05
        mu = f(X)
        a, b = -mu/sigma, (1-mu)/sigma
        Phi_a, Phi_b = scipy.stats.norm.cdf(a), scipy.stats.norm.cdf(b)
        phi_a, phi_b = scipy.stats.norm.pdf(a), scipy.stats.norm.pdf(b)
        return mu*(Phi_b-Phi_a)+sigma*(phi_a-phi_b)+1-Phi_b
    return clip_f

z = clip(f)(combine)
combine["y"] = z.values
FEATURES.append("y")

def clip: 0-1의 정규분포 확률로 만들어주는 함수  



## - Identify Nums and Cats

In [ ]:
CATS = []
NUMS = []
for c in FEATURES:
    t = "CAT"
    if combine[c].dtype=='object':
        CATS.append(c)
    else:
        NUMS.append(c)
        t = "NUM"
    n = combine[c].nunique()
    na = combine[c].isna().sum()
    print(f"[{t}] {c} has {n} unique and {na} NA")
print("CATS:", CATS )
print("NUMS:", NUMS )

## - Label Encode Cats

In [ ]:
SIZES = {}
for c in CATS:
    combine[c],_ = combine[c].factorize()
    SIZES[c] = combine[c].max()+1
    combine[c] = combine[c].astype('int32')
    combine[c] = combine[c].astype('int32')
print("Cardinality of all CATS:", SIZES )

In [ ]:
train = combine.iloc[:len(train)]
test = combine.iloc[len(train):len(train)+len(test)]
orig = combine.iloc[-len(orig):]
print(f"Train shape: {train.shape}, Test shape: {test.shape}, Original data shape: {orig.shape}")

## - Target Encode

In [ ]:
TE = []
for c in FEATURES:
    tmp = orig.groupby(c)[TARGET].mean()
    n = f"TE_{c}"
    print(f"{n}, ",end="")
    tmp.name = n
    train = train.merge(tmp, on=c, how='left')
    test = test.merge(tmp, on=c, how='left')
    TE.append(n)

# Train XGBoost on Residuals
We will train XGB on residuals. Instead of using `accident_risk` as target, we will use `target = accident_risk - y` where `y` is @siukeitin (Kaggle user broccoli beef) optimal original data solution from [here][1]

[1]: https://www.kaggle.com/competitions/playground-series-s5e10/discussion/609994#3296622

In [ ]:
from sklearn.model_selection import KFold
import xgboost as xgb

print(f"XGBoost version {xgb.__version__}")

In [ ]:
FOLDS = 7
SEED = 42

params = {
    "objective": "reg:squarederror",   
    "eval_metric": "rmse",             
    "learning_rate": 0.01,
    "max_depth": 6,                    
    "subsample": 0.9,
    "colsample_bytree": 0.6,
    "seed": SEED,
    "device": "cuda",
}

In [ ]:
oof_preds = np.zeros(len(train))
test_preds = np.zeros(len(test))

kf = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
for fold, (train_idx, val_idx) in enumerate(kf.split(train)):
    print("#"*25)
    print(f"### Fold {fold+1} ###")
    print("#"*25)

    X_train = train.iloc[train_idx][FEATURES+TE].copy()
    y_train = train.iloc[train_idx][TARGET] - train.iloc[train_idx]['y']
    
    X_valid = train.iloc[val_idx][FEATURES+TE].copy()
    y_valid = train.iloc[val_idx][TARGET] - train.iloc[val_idx]['y']
    y_valid2 = train.iloc[val_idx]['y'].values
    
    X_test = test[FEATURES+TE].copy()
    y_test2 = test['y'].values
        
    dtrain = xgb.DMatrix(X_train, label=y_train, enable_categorical=True)
    dval   = xgb.DMatrix(X_valid, label=y_valid, enable_categorical=True)
    dtest  = xgb.DMatrix(X_test, enable_categorical=True)

    model = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=100_000,
        evals=[(dtrain, "train"), (dval, "valid")],
        early_stopping_rounds=200,
        verbose_eval=200
    )

    oof_preds[val_idx] = model.predict(dval, iteration_range=(0, model.best_iteration + 1)) +y_valid2
    test_preds += (model.predict(dtest, iteration_range=(0, model.best_iteration + 1)) +y_test2)/ FOLDS

# CV Score
The first CV score below is our XGB model which has improved upon the optimal original data solution (by training on residuals). The second CV score below is using optimal original data solution only.

In [ ]:
m = np.sqrt( np.mean( (oof_preds - train[TARGET].values)**2. ) )
print(f" Overall CV RMSE = {m}")
np.save(f"oof",oof_preds)

In [ ]:
m = np.sqrt( np.mean( (train.y.values - train[TARGET].values)**2. ) )
print(f" Baseline CV RMSE = {m}")

# OOF EDA
We plot true vs predicted below. Discussion from @tilii7 (Kaggle user Tilii) about this plot is [here][1]

[1]: https://www.kaggle.com/competitions/playground-series-s5e10/discussion/610422

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(train[TARGET].values,oof_preds,s=0.25)
plt.plot([0,1],[0,1],'--',color='black')
plt.title("True vs Predicted")
plt.xlabel("True Target")
plt.ylabel("Predicted Target")
plt.show()

# XGB Feature Importance

In [ ]:
plt.rcParams["figure.dpi"] = 160      
fig, ax = plt.subplots(figsize=(15, 12))

xgb.plot_importance(
    model,
    max_num_features=100,
    importance_type="gain",
    ax=ax,
    show_values=False,                
    grid=False
)

ax.set_title("XGB Feature Importances", fontsize=18)
ax.tick_params(axis="both", labelsize=12)
fig.tight_layout()
plt.show()

# Create Submission CSV

In [ ]:
sub = pd.read_csv("/kaggle/input/playground-series-s5e10/sample_submission.csv")
sub['accident_risk'] = test_preds
sub.to_csv("submission.csv",index=False)
sub.head()

# Test Pred EDA

In [ ]:
plt.hist(sub['accident_risk'],bins=100)
plt.title("Histogram of Test Preds")
plt.show()